In [ ]:
from random import shuffle
import json
import os


class Card:
    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit

    def __str__(self):
        return f"{self.rank} of {self.suit}"


class Deck:
    def __init__(self):
        suits = ['hearts', 'diamonds', 'clubs', 'spades']
        ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'jack', 'queen', 'king', 'ace']
        self.cards = [Card(rank, suit) for suit in suits for rank in ranks]
        shuffle(self.cards)

    def deal(self):
        return self.cards.pop()


class Player:
    def __init__(self, name):
        self.name = name
        self.hand = []
        self.chips = 100  # Starting chips
        self.current_bet = 0

    def bet(self, amount):
        if amount > self.chips:
            raise ValueError("Not enough chips to bet.")
        self.chips -= amount
        self.current_bet += amount

    def reset_bet(self):
        self.current_bet = 0

    def receive_card(self, card):
        self.hand.append(card)

    def show_hand(self):
        return ', '.join(str(card) for card in self.hand)

    def __str__(self):
        return f"{self.name} has {self.chips} chips."


class AIPlayer(Player):
    def __init__(self, name):
        super().__init__(name)

    def decide_action(self, current_bet):
        if self.current_bet < current_bet:
            # Simple AI logic for betting
            if self.chips >= current_bet:
                return current_bet  # Call the current bet
            else:
                return 0  # Fold if not enough chips
        return self.current_bet  # Check if current bet is matched


class PokerGame:
    def __init__(self, player_names):
        self.deck = Deck()
        self.players = [Player(name) for name in player_names]
        self.players.append(AIPlayer("AI"))  # Add an AI player
        self.community_cards = []
        self.pot = 0
        self.current_bet = 0
        self.save_file = "poker_game_state.json"

    def betting_round(self):
        for player in self.players:
            print(f"\n{player.name}, it's your turn. You have {player.chips} chips.")
            if isinstance(player, AIPlayer):
                action = player.decide_action(self.current_bet)
                print(f"{player.name} decides to {'call' if action == self.current_bet else 'fold'}.")
                if action == 0:  # AI folds
                    print(f"{player.name} folds.")
                    self.players.remove(player)
                    continue
            else:
                action = self.get_bet(player)
                if action == 0:  # Player folded
                    print(f"{player.name} folds.")
                    self.players.remove(player)
                    continue

            player.bet(action)
            self.pot += action
            self.current_bet = max(self.current_bet, player.current_bet)

    def get_bet(self, player):
        while True:
            try:
                action = input(f"{player.name}, choose action - Bet/Raise/Call/Check/Fold: ").lower()
                if action == 'fold':
                    return 0  # Indicates folding
                elif action == 'check':
                    if player.current_bet < self.current_bet:
                        print("You cannot check; you must match the current bet.")
                        continue
                    return player.current_bet  # Match current bet
                elif action in ['bet', 'raise']:
                    amount = int(input(f"Enter your bet amount: "))
                    if amount < 0 or amount > player.chips:
                        print("Invalid bet. Please try again.")
                    else:
                        return amount
                else:
                    print("Invalid action. Please try again.")
            except ValueError:
                print("Invalid input. Please enter a valid number.")

    def deal_hole_cards(self):
        for player in self.players:
            player.receive_card(self.deck.deal())
            player.receive_card(self.deck.deal())
            print(f"{player.name} has {player.show_hand()}")

    def deal_community_cards(self, num_cards):
        for _ in range(num_cards):
            card = self.deck.deal()
            self.community_cards.append(card)

    def show_community_cards(self):
        print("Community Cards: " + ', '.join(str(card) for card in self.community_cards))

    def evaluate_hands(self):
        hand_strengths = []
        for player in self.players:
            hand_strength = self.get_hand_rank(player.hand + self.community_cards)
            hand_strengths.append((player, hand_strength))
        return hand_strengths

    def get_hand_rank(self, hand):
        values = [self.get_card_value(card.rank) for card in hand]
        suits = [card.suit for card in hand]

        unique_values = set(values)
        is_flush = len(set(suits)) == 1
        is_straight = len(unique_values) == 5 and (max(unique_values) - min(unique_values) == 4)

        if is_flush and is_straight:
            return (8, max(values))  # Straight flush
        if len(unique_values) == 2:
            return (7, max(values))  # Four of a kind or full house
        if is_flush:
            return (5, max(values))  # Flush
        if is_straight:
            return (4, max(values))  # Straight
        if len(unique_values) == 3:
            return (3, max(values))  # Three of a kind or two pair
        if len(unique_values) == 4:
            return (2, max(values))  # One pair
        return (1, max(values))  # High card

    def get_card_value(self, rank):
        values = {
            '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7,
            '8': 8, '9': 9, '10': 10, 'jack': 11, 'queen': 12, 'king': 13, 'ace': 14
        }
        return values[rank]

    def determine_winner(self, hand_strengths):
        winner = max(hand_strengths, key=lambda x: x[1])
        print(f"\nWinner: {winner[0].name} with hand strength: {winner[1]}!")
        winner[0].chips += self.pot  # Winner takes the pot

    def save_game_state(self):
        state = {
            'players': [{'name': player.name, 'chips': player.chips} for player in self.players],
            'pot': self.pot,
            'community_cards': [str(card) for card in self.community_cards]
        }
        with open(self.save_file, 'w') as f:
            json.dump(state, f)
        print("Game state saved!")

    def load_game_state(self):
        if os.path.exists(self.save_file):
            with open(self.save_file, 'r') as f:
                state = json.load(f)
                for player_data in state['players']:
                    for player in self.players:
                        if player.name == player_data['name']:
                            player.chips = player_data['chips']
                self.pot = state['pot']
                self.community_cards = [Card(*card.split(' of ')) for card in state['community_cards']]
                print("Game state loaded!")
        else:
            print("No saved game state found.")

    def play(self):
        print("Welcome to Poker!")
        self.load_game_state()  # Load previous game state if it exists
        while True:
            self.community_cards = []  # Reset community cards and pot for the new round
            self.pot = 0
            self.current_bet = 0
            self.deck = Deck()  # Create a new deck for each round

            self.deal_hole_cards()
            self.betting_round()

            self.deal_community_cards(3)  # Flop
            self.show_community_cards()
            self.betting_round()

            self.deal_community_cards(1)  # Turn
            self.show_community_cards()
            self.betting_round()

            self.deal_community_cards(1)  # River
            self.show_community_cards()

            hand_strengths = self.evaluate_hands()
            self.determine_winner(hand_strengths)

            if input("Do you want to play another round? (yes/no): ").lower() != "yes":
                self.save_game_state()  # Save the game state before exiting
                break

        print("Thanks for playing!")


# Start the game
if __name__ == "__main__":
    player_names = input("Enter player names (comma separated): ").split(",")
    poker_game = PokerGame([name.strip() for name in player_names])
    poker_game.play()


<style>
    /* Make the entire page background black */
    body {
        background-color: #000000 !important; /* Black page background */
        color: #ffffff !important; /* Default white text color */
    }

    article {
        background-color: #000000 !important; /* Black background for article */
        border: 3px solid #4b0082 !important; /* Dark purple border */
        padding: 25px !important;
        border-radius: 50px !important;
    }

    article h1 {
        color: #ffffff !important; /* White header */
    }

    article h2, h3, h4, p {
        color: #ffffff !important; /* White text */
    }

    article ul, ol, li {
        background-color: #4b0082 !important; /* Dark purple for lists */
        border-left: 5px solid #800080 !important; /* Slightly lighter purple list border */
        color: #ffffff !important; /* White list text */
        padding: 12px 25px !important;
        margin: 12px 0 !important;
        border-radius: 50px !important;
    }

    article .center-text {
        text-align: center !important;
    }

    article summary {
        color: #ffffff !important; /* White summary */
    }

    article code {
        color: #ffffff !important; /* White code text */
        background-color: #800080 !important; /* Darker purple background for code */
        padding: 3px 6px !important;
        border-radius: 50px !important;
    }
</style>
